In [1]:
#encoding=utf-8
import sys, codecs, re, os, csv
from io import open
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import itertools

pd.options.display.max_rows = 9999999
path = os.getcwd()  
AppleDir = os.path.join(path,"AppleAds")
OriDir = os.path.join(path,"OriAds")

In [3]:
AppleDir

'/Users/onyi/Dropbox/newspaper/advertisement/AppleAds'

<h4> Reading the Advertisement</h4>
The ads are stored as txt files with a few structure. I parse the txt file to retrive relvenat information (company name, product name, date...etc), and then convert them into dataframe for analysis

In [3]:
OriIndList = {}
OriComp ={}
OriAd={}
OriCompiled=DataFrame()
 

#Read Advertisement at Orietal Daily
for dir_entry in os.listdir(OriDir):
    if not dir_entry.startswith('.'):
        dir_entry_path = os.path.join(OriDir, dir_entry)
        if os.path.isfile(dir_entry_path):
            try: 
                with open(dir_entry_path) as f: 
                    RawOriInd = f.read()
                    OriIndList[dir_entry] = re.split(r': Wisers electronic service. This content, the trademarks and logos belong to Wisers, the relevant organizations or copyright owners. All rights reserved. Any content provided by user is the responsibility of the user and Wisers is not responsible for such content, copyright clearance or any damage/loss suffered as a result.', RawOriInd)
                    OriIndList[dir_entry] = [ re.sub(r'\n', ' ', ad) for ad in OriIndList[dir_entry]]
                    OriIndList[dir_entry] = [ re.sub(r'--', ' ', ad) for ad in OriIndList[dir_entry]]
                    OriIndList[dir_entry] = [ re.sub(r'\s+', ' ', ad) for ad in OriIndList[dir_entry]]
                    OriIndList[dir_entry] = [ re.sub(r'- Source|Newspaper', ' ', ad) for ad in OriIndList[dir_entry]]
                    OriIndList[dir_entry] = [ re.split(r'廣告:|Product Category:| \| | DOCUMENT ID: | Brand:' , ad) for ad in OriIndList[dir_entry] ]      
                    OriAd[dir_entry] = DataFrame(OriIndList[dir_entry])
                    OriAd[dir_entry]= OriAd[dir_entry].ix[:,0:8]
                    OriAd[dir_entry].columns = [ 'newspaper','date','position','x','x2','ad','industries','Company','ID']
                    OriAd[dir_entry]['Company'] = OriAd[dir_entry]['Company'].str.split('Product:').apply(Series, 1)[0]
                    ind = re.sub(r".txt", '', dir_entry)
                    ind_name=ind[7:]
                    OriAd[dir_entry]['industry'] = ind_name
                    OriAd[dir_entry] = OriAd[dir_entry][OriAd[dir_entry].Company.str.contains("東方日報") == False ]
                    
                
                    #spliting the companies into rows
                    s = OriAd[dir_entry]['Company'].str.split(',').apply(Series, 1).stack()
                    s.index = s.index.droplevel(-1)
                    s.name = 'Company'
                    del OriAd[dir_entry]['Company']
                    OriAd[dir_entry]=OriAd[dir_entry].join(s)
                    
                    #Discard NaN rows
                    OriAd[dir_entry] = OriAd[dir_entry][pd.notnull(OriAd[dir_entry].Company)] #remove entries with nan company
                    #Discard companies == ltd.
                    OriAd[dir_entry] = OriAd[dir_entry][OriAd[dir_entry].Company != 'Ltd.' ]
                    OriAd[dir_entry] = OriAd[dir_entry][OriAd[dir_entry].Company != 'ltd.' ]
                    
                    
                    #write the dataframe into CSV file
                    name = re.sub(r".txt", '', dir_entry)+'df.csv'
                    OriDirDf = OriDir+'/Df'
                    OriAd[dir_entry].to_csv(os.path.join(OriDirDf, name))
                        
            except UnicodeDecodeError as e: 
                    print("Some error occurred decoding file %s: %s" % (dir_entry, e)) 


In [4]:
AppleIndList = {}
ApAd = {}
ApCompiled=DataFrame()
ApComp = {}

#Read Advertisement at Apple Daily
for dir_entry in os.listdir(AppleDir):
    if not dir_entry.startswith('.'):
        dir_entry_path = os.path.join(AppleDir, dir_entry)
        if os.path.isfile(dir_entry_path):
            try: 
                with open(dir_entry_path) as f: 
                    RawApInd = f.read()
                    AppleIndList[dir_entry] = re.split(r': Wisers electronic service. This content, the trademarks and logos belong to Wisers, the relevant organizations or copyright owners. All rights reserved. Any content provided by user is the responsibility of the user and Wisers is not responsible for such content, copyright clearance or any damage/loss suffered as a result.', RawApInd)
                    AppleIndList[dir_entry] = [ re.sub(r'\n', ' ', ad) for ad in AppleIndList[dir_entry]]
                    AppleIndList[dir_entry] = [ re.sub(r'--', ' ', ad) for ad in AppleIndList[dir_entry]]
                    AppleIndList[dir_entry] = [ re.sub(r'\s+', ' ', ad) for ad in AppleIndList[dir_entry]]
                    AppleIndList[dir_entry] = [ re.sub(r'- Source|Newspaper', ' ', ad) for ad in AppleIndList[dir_entry]]
                    AppleIndList[dir_entry] = [ re.split(r'廣告:|Product Category:| \| | DOCUMENT ID: | Brand:' , ad) for ad in AppleIndList[dir_entry] ]
 
                    ApAd[dir_entry] = DataFrame(AppleIndList[dir_entry])
                    ApAd[dir_entry]= ApAd[dir_entry].ix[:,0:8]
                    ApAd[dir_entry].columns = ['newspaper', 'date','position','x','x2','ad','industries','Company','ID']
                    ApAd[dir_entry]['Company'] = ApAd[dir_entry]['Company'].str.split('Product:').apply(Series, 1)[0]
                    ind = re.sub(r".txt", '', dir_entry)
                    ind_name=ind[9:]
                    ApAd[dir_entry]['industry'] = ind_name  
                    
                    ApAd[dir_entry] = ApAd[dir_entry][ApAd[dir_entry].Company.str.contains("蘋果日報") == False ]
                
                    #spliting the companies into rows
                    s = ApAd[dir_entry]['Company'].str.split(',').apply(Series, 1).stack()
                    s.index = s.index.droplevel(-1)
                    s.name = 'Company'
                    del ApAd[dir_entry]['Company']
                    ApAd[dir_entry]=ApAd[dir_entry].join(s)
                    
                    #Discard NaN rows
                    ApAd[dir_entry] = ApAd[dir_entry][pd.notnull(ApAd[dir_entry].Company)] #remove entries with nan company
                   
                    #Discard companies == ltd.
                    ApAd[dir_entry] = ApAd[dir_entry][ApAd[dir_entry].Company != 'Ltd.' ]
                    ApAd[dir_entry] = ApAd[dir_entry][ApAd[dir_entry].Company != 'ltd.' ]
     
                    #write the dataframe into CSV file
                    name = re.sub(r".txt", '', dir_entry)+'df.csv'
                    ApDirDf = AppleDir+'/Df'
                    ApAd[dir_entry].to_csv(os.path.join(ApDirDf, name))
                    
                    
                    
            except UnicodeDecodeError as e: 
                    print("Some error occurred decoding file %s: %s" % (dir_entry, e)) 


<h4> Getting a list of Unique Companies </h4>
I need to get a list of unique companies to determine the company charcaterisitcs (foreign vs local, politically connected or not)

In [8]:
# sort by company names
OriComp=OriCompiled['companies'].value_counts().sort_index()
ApComp = ApCompiled['companies'].value_counts().sort_index()

#append all companies together and export to csv
writer = csv.writer(open('OriCompList.csv', 'wt'),  delimiter=',')
for value in OriComp.items():
    writer.writerow([value])

writer = csv.writer(open('ApCompList.csv', 'wt'))
for value in ApComp.items():
    writer.writerow([value])

In [4]:
#read csv file of all the companies
with open('AllCompanies.csv', 'rt') as f:
    reader = csv.reader(f)
    CompaniesList = list(reader)


In [5]:
#flatten the list of list to list
import functools
CompList=functools.reduce(lambda x,y: x+y,CompaniesList)

#remove the products in the companaies 
for index, item in enumerate(CompList):
    
    found=re.search('(.+?)Product:', item)
    if found is not None:
        found=re.search('(.+?) Product:', item).group(1)
        CompList[index] = found
    #if len(found)>0:
    #re.sub(r'(.+?)Product:(.+?)', found, )
        


In [130]:
#function to retrieve unique values in a list
def UniqueVal(seq, idfun=None): 
    # order preserving
    if idfun is None:
        def idfun(x): return x
   
    seen = {}
    result = []
    for item in seq:
        marker = idfun(item)
        if marker in seen: continue
        seen[marker] = 1
        result.append(item)
    return result

In [131]:
UniqComp=UniqueVal(CompList)

writer = csv.writer(open('UniqCompList.csv', 'wt'),  delimiter=',')
for value in UniqComp:
    writer.writerow([value])